# PowerBI 工具包

本笔记本展示了一个与 `Power BI Dataset` 交互的代理。该代理能够回答关于数据集的一般性问题，并且可以从错误中恢复。

请注意，由于此代理尚处于积极开发阶段，所有答案可能并不完全正确。它通过 [executequery endpoint](https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/execute-queries) 运行，该端点不允许删除操作。

### 注意事项：
- 它依赖于 `azure.identity` 包进行身份验证，可以使用 `pip install azure-identity` 进行安装。或者，您也可以使用字符串令牌创建 Power BI 数据集，而无需提供凭据。
- 您还可以提供一个用户名，用于模拟启用 RLS 的数据集。
- 该工具包使用 LLM 从问题中创建查询，代理则使用 LLM 进行整体执行。
- 测试主要针对 `gpt-3.5-turbo-instruct` 模型进行，codex 模型表现似乎不佳。

## 初始化

In [ ]:
from azure.identity import DefaultAzureCredential
from langchain_community.agent_toolkits import PowerBIToolkit, create_pbi_agent
from langchain_community.utilities.powerbi import PowerBIDataset
from langchain_openai import ChatOpenAI

In [ ]:
fast_llm = ChatOpenAI(
    temperature=0.5, max_tokens=1000, model_name="gpt-3.5-turbo", verbose=True
)
smart_llm = ChatOpenAI(temperature=0, max_tokens=100, model_name="gpt-4", verbose=True)

toolkit = PowerBIToolkit(
    powerbi=PowerBIDataset(
        dataset_id="<dataset_id>",
        table_names=["table1", "table2"],
        credential=DefaultAzureCredential(),
    ),
    llm=smart_llm,
)

agent_executor = create_pbi_agent(
    llm=fast_llm,
    toolkit=toolkit,
    verbose=True,
)

## 示例：描述表格

In [ ]:
agent_executor.run("Describe table1")

## 示例：查询表中的简单数据
在此示例中，代理程序实际能够找出正确的查询，以获取表的行数。

In [ ]:
agent_executor.run("How many records are in table1?")

## 示例：运行查询

In [ ]:
agent_executor.run("How many records are there by dimension1 in table2?")

In [ ]:
agent_executor.run("What unique values are there for dimensions2 in table2")

## 示例：添加您自己的少量样本提示

In [ ]:
# fictional example
few_shots = """
Question: How many rows are in the table revenue?
DAX: EVALUATE ROW("Number of rows", COUNTROWS(revenue_details))
----
Question: How many rows are in the table revenue where year is not empty?
DAX: EVALUATE ROW("Number of rows", COUNTROWS(FILTER(revenue_details, revenue_details[year] <> "")))
----
Question: What was the average of value in revenue in dollars?
DAX: EVALUATE ROW("Average", AVERAGE(revenue_details[dollar_value]))
----
"""
toolkit = PowerBIToolkit(
    powerbi=PowerBIDataset(
        dataset_id="<dataset_id>",
        table_names=["table1", "table2"],
        credential=DefaultAzureCredential(),
    ),
    llm=smart_llm,
    examples=few_shots,
)
agent_executor = create_pbi_agent(
    llm=fast_llm,
    toolkit=toolkit,
    verbose=True,
)

In [ ]:
agent_executor.run("What was the maximum of value in revenue in dollars in 2022?")